# Part III — Real Projects  
## 16. Project 1 (Beginner): Blog / Content Site (End-to-End Build)

You already learned the building blocks (URLs, views, templates, models, ORM, forms,
auth, middleware, static/media, email). Now you’ll **assemble them into a coherent,
industry-standard mini product**: a content site.

This project is designed to feel like what you would build in a real company for:
- a marketing blog
- documentation/news site
- internal knowledge base
- editorial publishing tool

We will build (with explanations + code):
- public article listing (search + tag filter + pagination)
- public article detail page with cover image + SEO meta
- draft visibility rules (public sees only published; authors/staff can preview)
- comments (optional for public, moderated in admin)
- admin editorial workflow improvements
- sitemap + robots.txt basics

You can treat this as a “complete mini app” you can show in a portfolio.

---

## 16.0 Outcomes (What you will have at the end)

### Visitor features
- Browse articles on `/articles/`
- Filter by tag: `/articles/?tag=django`
- Search: `/articles/?q=templates`
- Paginate: `/articles/?page=2`
- Open an article: `/articles/hello-django/`
- See tags and (optionally) comments on article detail

### Author/editor features
- Log in
- Create and edit articles
- Draft/publish workflow
- Cover image upload
- Receive email notification on publish (from Chapter 15)

### Admin/editorial features
- Manage articles and tags
- Moderate comments (approve/hide)
- See helpful columns and filters

### SEO/ops features
- `/sitemap.xml`
- `/robots.txt`
- Stable canonical URLs via `get_absolute_url()`

---

## 16.1 Requirements and User Stories (Write These Down)

### Public site
1. As a visitor, I can see a list of published articles ordered by newest.
2. As a visitor, I can search articles by title/body.
3. As a visitor, I can filter articles by tag.
4. As a visitor, I can paginate through articles.
5. As a visitor, I can open an article and see its cover image, body, tags, and publish date.

### Auth + editorial workflow
6. As a logged-in user, I can create an article (default draft).
7. As the author, I can edit my own article.
8. As staff, I can edit any article.
9. As a visitor, I cannot view drafts (should be 404 or 403 depending on policy).
10. As an author/staff, I can preview drafts via the normal detail URL.

### Comments (beginner-friendly moderation workflow)
11. As a visitor, I can submit a comment to an article (requires name + body).
12. As an admin/staff, I can approve comments before they appear publicly.
13. As a visitor, I only see approved comments.

### SEO basics
14. As a search engine, I can discover site URLs via sitemap.
15. Robots can access public pages; admin pages are disallowed in robots.txt.

---

## 16.2 Route Map (Industry-Standard URL Design)

We’ll design a clean, stable URL structure:

- Article list: `GET /articles/`
- Article detail: `GET /articles/<slug>/`
- Article create: `GET/POST /articles/new/`
- Article edit: `GET/POST /articles/<slug>/edit/`
- Comment create: `POST /articles/<slug>/comments/`
- Sitemap: `GET /sitemap.xml`
- Robots: `GET /robots.txt`

### Why this structure is standard
- `/articles/` is the “collection”
- `/articles/<slug>/` is a “resource”
- `new/` and `edit/` are explicit actions (kept more specific than `<slug>`)

---

## 16.3 Model Improvements: Add `get_absolute_url()` (SEO + Reuse)

A very common Django convention is to define `get_absolute_url()` on models so
templates and sitemap generation can rely on it.

Edit `articles/models.py` (inside `Article`):

```python
from django.urls import reverse

class Article(models.Model):
    ...
    def get_absolute_url(self) -> str:
        return reverse("articles:detail", kwargs={"slug": self.slug})
```

### Why this matters
- Templates can use `{{ article.get_absolute_url }}` without knowing route names.
- Sitemaps can use it automatically.
- If you ever change URLs, you update one place.

Now we’ll standardize route names to match that.

---

## 16.4 Refactor URLs to a Clean Public Interface

Right now your app likely has mixed `html/` paths. For a real content site, make
HTML the default and (optionally) move JSON to `/api/` later.

### 16.4.1 Update `articles/urls.py`
Replace with:

```python
from django.urls import path

from . import views

app_name = "articles"

urlpatterns = [
    path("", views.article_list, name="list"),
    path("new/", views.article_create, name="create"),
    path("<slug:slug>/edit/", views.article_edit, name="edit"),
    path("<slug:slug>/comments/", views.comment_create, name="comment_create"),
    path("<slug:slug>/", views.article_detail, name="detail"),
]
```

### Important ordering note
- `new/` and `<slug>/edit/` must come before `<slug>/` because `<slug>` would match
  “new” otherwise.

### 16.4.2 Update navigation links
Edit `templates/partials/_nav.html` accordingly:

```django
<a href="{% url 'articles:list' %}">Articles</a>
<a href="{% url 'articles:create' %}">New Article</a>
```

---

## 16.5 Query Layer: A “Published” QuerySet (Clean, Reusable)

This is an industry pattern that keeps “published-only” logic consistent.

### 16.5.1 Add a custom QuerySet
Edit `articles/models.py`:

```python
from django.db import models

class ArticleQuerySet(models.QuerySet):
    def published(self):
        return self.filter(status=Article.Status.PUBLISHED)

    def visible_to(self, user):
        """
        Public sees only published.
        Staff sees everything.
        Author sees their own drafts too.
        """
        if user.is_authenticated and user.is_staff:
            return self

        if user.is_authenticated:
            return self.filter(
                models.Q(status=Article.Status.PUBLISHED)
                | models.Q(author=user)
            )

        return self.published()
```

Then in `Article` model, add:

```python
class Article(models.Model):
    ...
    objects = ArticleQuerySet.as_manager()
```

### Explanation
- `Article.objects.published()` becomes a readable, consistent query.
- `visible_to(user)` centralizes access rules and avoids duplicating logic in every
  view. This is how professional Django apps stay consistent.

---

## 16.6 Build Public Views (List + Detail) With Search/Filter/Pagination

We’ll implement:
- list page with query params: `q`, `tag`, `page`
- detail page with access control (published unless owner/staff)
- tag display

### 16.6.1 Create a GET “filter form” (clean parsing + validation)
Create `articles/forms_public.py` (or put in `articles/forms.py` if you prefer):

```python
from django import forms


class ArticleListFilterForm(forms.Form):
    q = forms.CharField(required=False, max_length=100)
    tag = forms.CharField(required=False, max_length=60)
    page = forms.IntegerField(required=False, min_value=1)

    def clean_q(self):
        q = self.cleaned_data.get("q") or ""
        return q.strip()

    def clean_tag(self):
        tag = self.cleaned_data.get("tag") or ""
        return tag.strip()
```

#### Why use a Form for GET query params?
Because query params are still user input. Forms give you:
- parsing (page becomes int)
- validation (page must be >= 1)
- normalized cleaned data
- consistent error handling

This is a professional pattern, not “overkill.”

### 16.6.2 Implement list view with Paginator
Edit `articles/views.py` (replace/merge with your existing views):

```python
from django.core.paginator import EmptyPage, PageNotAnInteger, Paginator
from django.db.models import Q
from django.shortcuts import render
from django.views.decorators.http import require_GET

from .forms_public import ArticleListFilterForm
from .models import Article


@require_GET
def article_list(request):
    form = ArticleListFilterForm(request.GET)
    form.is_valid()  # even if invalid, we’ll fall back safely

    q = form.cleaned_data.get("q") if form.is_valid() else (request.GET.get("q") or "")
    tag = form.cleaned_data.get("tag") if form.is_valid() else (request.GET.get("tag") or "")
    page_number = form.cleaned_data.get("page") if form.is_valid() else 1

    qs = Article.objects.visible_to(request.user).select_related("author").prefetch_related("tags")

    # Public list: usually show only published (unless you want “my drafts” mixed in).
    # For a true public blog, enforce published-only:
    qs = qs.published()

    if tag:
        qs = qs.filter(tags__slug=tag)

    if q:
        qs = qs.filter(Q(title__icontains=q) | Q(body__icontains=q))

    qs = qs.distinct().order_by("-published_at", "-created_at")

    paginator = Paginator(qs, 10)  # 10 articles per page

    try:
        page_obj = paginator.page(page_number or 1)
    except PageNotAnInteger:
        page_obj = paginator.page(1)
    except EmptyPage:
        page_obj = paginator.page(paginator.num_pages)

    return render(
        request,
        "articles/list.html",
        {
            "form": form,
            "page_obj": page_obj,
            "paginator": paginator,
            "articles": page_obj.object_list,
            "q": q,
            "tag": tag,
        },
    )
```

#### Deep explanation of pagination mechanics (why it’s done like this)
- We use `Paginator(qs, 10)` so the DB query becomes `LIMIT/OFFSET`.
- `page_obj` contains:
  - the objects for this page
  - metadata: has_next/has_previous, next_page_number, etc.
- We handle:
  - non-integer page (`?page=abc`) → page 1
  - out-of-range page (`?page=999`) → last page
This makes your app resilient to bad inputs and bots.

#### Why keep filters in query string
Because list state should be:
- bookmarkable
- shareable
- compatible with browser navigation/back button
That’s why search/filter/pagination typically use GET.

---

### 16.6.3 Implement detail view with visibility rules
Edit `articles/views.py`:

```python
from django.http import Http404
from django.shortcuts import get_object_or_404, render
from django.views.decorators.http import require_GET

from .models import Article


@require_GET
def article_detail(request, slug: str):
    # Visibility policy:
    # - Published articles are visible to everyone.
    # - Drafts are visible only to author or staff.
    qs = Article.objects.visible_to(request.user).select_related("author").prefetch_related("tags")

    article = get_object_or_404(qs, slug=slug)

    # If you want to hide drafts completely from non-author by returning 404,
    # visible_to already handles that. If you prefer 403, you’d do permission checks.

    comments_qs = article.comments.filter(is_approved=True).order_by("created_at")

    return render(
        request,
        "articles/detail.html",
        {
            "article": article,
            "comments": comments_qs,
        },
    )
```

---

## 16.7 Comments: Add a Comment Model + Moderation

### 16.7.1 Add `Comment` model
Edit `articles/models.py`:

```python
from django.core.validators import MinLengthValidator

class Comment(models.Model):
    article = models.ForeignKey(
        Article,
        on_delete=models.CASCADE,
        related_name="comments",
    )
    name = models.CharField(max_length=80)
    body = models.TextField(validators=[MinLengthValidator(5)])
    is_approved = models.BooleanField(default=False)
    created_at = models.DateTimeField(auto_now_add=True)

    class Meta:
        ordering = ["created_at"]
        indexes = [
            models.Index(fields=["article", "is_approved", "created_at"]),
        ]

    def __str__(self) -> str:
        return f"Comment by {self.name} on {self.article.slug}"
```

#### Why `is_approved` is important
Public comments are a spam magnet. A beginner-friendly but real workflow is:
- accept comment submission
- keep it unapproved by default
- staff approves via admin

Run migrations:

```bash
python manage.py makemigrations
python manage.py migrate
```

---

### 16.7.2 Create a CommentForm
Create `articles/forms_comments.py`:

```python
from django import forms

from .models import Comment


class CommentForm(forms.ModelForm):
    class Meta:
        model = Comment
        fields = ["name", "body"]
        widgets = {
            "body": forms.Textarea(attrs={"rows": 4}),
        }

    def clean_name(self):
        name = (self.cleaned_data.get("name") or "").strip()
        if len(name) < 2:
            raise forms.ValidationError("Name must be at least 2 characters.")
        return name
```

---

### 16.7.3 Add comment create view (POST-only + CSRF + PRG)
Edit `articles/views.py`:

```python
from django.contrib import messages
from django.shortcuts import redirect
from django.views.decorators.http import require_POST

from .forms_comments import CommentForm
from .models import Article, Comment


@require_POST
def comment_create(request, slug: str):
    article = get_object_or_404(
        Article.objects.published(),
        slug=slug,
    )

    form = CommentForm(request.POST)
    if form.is_valid():
        comment = form.save(commit=False)
        comment.article = article
        comment.is_approved = False  # moderation default
        comment.save()
        messages.success(request, "Thanks! Your comment was submitted for review.")
    else:
        messages.error(request, "Please correct the errors in your comment.")

        # If invalid, re-render article detail with errors (no redirect),
        # because redirect would lose field-level error display.
        comments_qs = article.comments.filter(is_approved=True)
        return render(
            request,
            "articles/detail.html",
            {
                "article": article,
                "comments": comments_qs,
                "comment_form": form,
            },
            status=400,
        )

    return redirect(article.get_absolute_url())
```

#### Why invalid comment returns 400 + re-renders
- You want to show field errors next to inputs.
- PRG is great for success, but for invalid forms re-rendering is standard.
- Returning 400 is correct: client sent invalid data.

---

### 16.7.4 Ensure detail view provides a blank comment form
Update `article_detail` to pass a form if not already present:

```python
from .forms_comments import CommentForm

@require_GET
def article_detail(request, slug: str):
    qs = Article.objects.visible_to(request.user).select_related("author").prefetch_related("tags")
    article = get_object_or_404(qs, slug=slug)

    comments_qs = article.comments.filter(is_approved=True).order_by("created_at")

    return render(
        request,
        "articles/detail.html",
        {
            "article": article,
            "comments": comments_qs,
            "comment_form": CommentForm(),
        },
    )
```

---

## 16.8 Templates: List, Detail, and Pagination UI (Reusable + Correct Links)

### 16.8.1 List template with search/filter form (GET)
Create/replace `articles/templates/articles/list.html`:

```django
{% extends "base.html" %}

{% block title %}Articles{% endblock %}

{% block content %}
  <h1>Articles</h1>

  <form method="get" class="filters">
    <div class="field">
      <label for="id_q">Search</label>
      <input id="id_q" type="text" name="q" value="{{ q|default:'' }}" />
    </div>

    <div class="field">
      <label for="id_tag">Tag</label>
      <input id="id_tag" type="text" name="tag" value="{{ tag|default:'' }}" />
      <small>Enter a tag slug like <code>django</code>.</small>
    </div>

    <button type="submit">Apply</button>

    {% if q or tag %}
      <a href="{% url 'articles:list' %}">Clear</a>
    {% endif %}
  </form>

  <hr />

  <ul>
    {% for a in articles %}
      <li>
        <h2>
          <a href="{{ a.get_absolute_url }}">{{ a.title }}</a>
        </h2>

        <p>
          by <strong>{{ a.author.username }}</strong>
          •
          {% if a.published_at %}{{ a.published_at|date:"Y-m-d" }}{% endif %}
        </p>

        {% if a.cover_image %}
          <img src="{{ a.cover_image.url }}" alt="Cover for {{ a.title }}" style="max-width: 240px;" />
        {% endif %}

        <p>{{ a.body|truncatechars:160 }}</p>

        <p>
          Tags:
          {% for t in a.tags.all %}
            <a href="{% url 'articles:list' %}?tag={{ t.slug }}">{{ t.name }}</a>
          {% empty %}
            <em>(no tags)</em>
          {% endfor %}
        </p>
      </li>
    {% empty %}
      <li>No articles found.</li>
    {% endfor %}
  </ul>

  {% include "articles/_pagination.html" with page_obj=page_obj q=q tag=tag %}
{% endblock %}
```

#### Why tag links use query params
Filtering is a “view of the collection,” so `?tag=...` is correct. It also composes:
`?tag=django&q=templates&page=2`.

---

### 16.8.2 Pagination partial (keeps query params)
Create `articles/templates/articles/_pagination.html`:

```django
{% if page_obj.paginator.num_pages > 1 %}
  <nav class="pagination">
    <p>
      Page {{ page_obj.number }} of {{ page_obj.paginator.num_pages }}
    </p>

    <div class="pager">
      {% if page_obj.has_previous %}
        <a href="?page=1{% if q %}&q={{ q|urlencode }}{% endif %}{% if tag %}&tag={{ tag|urlencode }}{% endif %}">
          First
        </a>
        <a href="?page={{ page_obj.previous_page_number }}{% if q %}&q={{ q|urlencode }}{% endif %}{% if tag %}&tag={{ tag|urlencode }}{% endif %}">
          Previous
        </a>
      {% endif %}

      {% if page_obj.has_next %}
        <a href="?page={{ page_obj.next_page_number }}{% if q %}&q={{ q|urlencode }}{% endif %}{% if tag %}&tag={{ tag|urlencode }}{% endif %}">
          Next
        </a>
        <a href="?page={{ page_obj.paginator.num_pages }}{% if q %}&q={{ q|urlencode }}{% endif %}{% if tag %}&tag={{ tag|urlencode }}{% endif %}">
          Last
        </a>
      {% endif %}
    </div>
  </nav>
{% endif %}
```

#### Important explanation: why we append `&q=...&tag=...`
If you don’t preserve filters while paginating:
- user searches “django”
- clicks “Next”
- filter disappears
This feels broken.

We preserve filters by re-attaching existing query params.

> Later (advanced), you can create a “querystring builder” template tag so you don’t
> repeat this pattern. For Project 1, explicit is fine and educational.

---

### 16.8.3 Detail template with comment form
Create/replace `articles/templates/articles/detail.html`:

```django
{% extends "base.html" %}

{% block title %}{{ article.title }}{% endblock %}

{% block content %}
  <article>
    <h1>{{ article.title }}</h1>

    <p>
      by <strong>{{ article.author.username }}</strong>
      {% if article.published_at %}
        • {{ article.published_at|date:"Y-m-d H:i" }}
      {% else %}
        • <em>Not published</em>
      {% endif %}
    </p>

    {% if article.cover_image %}
      <img
        src="{{ article.cover_image.url }}"
        alt="Cover for {{ article.title }}"
        style="max-width: 640px;"
      />
    {% endif %}

    <p>
      Tags:
      {% for t in article.tags.all %}
        <a href="{% url 'articles:list' %}?tag={{ t.slug }}">{{ t.name }}</a>
      {% empty %}
        <em>(no tags)</em>
      {% endfor %}
    </p>

    <hr />

    <div>
      {{ article.body|linebreaks }}
    </div>
  </article>

  <hr />

  <section>
    <h2>Comments</h2>

    <ul>
      {% for c in comments %}
        <li>
          <strong>{{ c.name }}</strong>
          <small>{{ c.created_at|date:"Y-m-d H:i" }}</small>
          <div>{{ c.body|linebreaksbr }}</div>
        </li>
      {% empty %}
        <li><em>No comments yet.</em></li>
      {% endfor %}
    </ul>

    <h3>Leave a comment</h3>

    <form method="post" action="{% url 'articles:comment_create' slug=article.slug %}">
      {% csrf_token %}

      {% if comment_form.non_field_errors %}
        <div class="form-errors">{{ comment_form.non_field_errors }}</div>
      {% endif %}

      <div class="field">
        <label for="{{ comment_form.name.id_for_label }}">Name</label>
        {{ comment_form.name }}
        {% for err in comment_form.name.errors %}<div class="error">{{ err }}</div>{% endfor %}
      </div>

      <div class="field">
        <label for="{{ comment_form.body.id_for_label }}">Comment</label>
        {{ comment_form.body }}
        {% for err in comment_form.body.errors %}<div class="error">{{ err }}</div>{% endfor %}
      </div>

      <button type="submit">Submit</button>

      <p><small>Comments appear after approval.</small></p>
    </form>
  </section>
{% endblock %}
```

#### Why `linebreaks` and `linebreaksbr` are used
- `linebreaks` turns newlines into `<p>` blocks (nice for article body).
- `linebreaksbr` turns newlines into `<br>` (good for comment text).

---

## 16.9 Update Create/Edit Article Forms to Match New Routes

Your `ArticleForm` already exists. Ensure create/edit views now redirect to the new
detail route name `articles:detail`.

In `articles/views.py` update redirects:

```python
return redirect("articles:detail", slug=article.slug)
```

Also ensure create/edit are still protected with `@login_required` and ownership
checks (from Chapter 12).

---

## 16.10 Admin: Moderate Comments + Inline View (Editorial UX)

### 16.10.1 Register Comment in admin
Edit `articles/admin.py`:

```python
from django.contrib import admin

from .models import Article, Comment, Tag


@admin.register(Comment)
class CommentAdmin(admin.ModelAdmin):
    list_display = ("id", "article", "name", "is_approved", "created_at")
    list_filter = ("is_approved", "created_at")
    search_fields = ("name", "body", "article__title", "article__slug")
    ordering = ("-created_at",)
    actions = ["approve_selected"]

    @admin.action(description="Approve selected comments")
    def approve_selected(self, request, queryset):
        queryset.update(is_approved=True)
```

### 16.10.2 Inline comments on Article edit page (optional but very useful)
Add inline in `ArticleAdmin`:

```python
class CommentInline(admin.TabularInline):
    model = Comment
    extra = 0
    fields = ("name", "body", "is_approved", "created_at")
    readonly_fields = ("created_at",)
```

Then in `ArticleAdmin`:

```python
@admin.register(Article)
class ArticleAdmin(admin.ModelAdmin):
    inlines = [CommentInline]
    # keep your existing list_display/search/filter configs
```

#### Why this is helpful
Editors can review comments directly while looking at the article.

---

## 16.11 SEO Basics: Sitemap + Robots + Meta Tags

### 16.11.1 Add sitemap framework
In `config/settings.py`, ensure:

```python
INSTALLED_APPS = [
    # ...
    "django.contrib.sites",     # optional; sitemap can work without it, but sites is common
    "django.contrib.sitemaps",
    # ...
]
```

If you add `sites`, also set:

```python
SITE_ID = 1
```

> If you don’t want Sites framework right now, you can still use sitemaps without it.
> For beginner projects, it’s fine either way. We’ll implement sitemap without relying
> on Sites.

Run migrations if you added sites:

```bash
python manage.py migrate
```

### 16.11.2 Create a sitemap class
Create `articles/sitemaps.py`:

```python
from django.contrib.sitemaps import Sitemap

from .models import Article


class ArticleSitemap(Sitemap):
    changefreq = "weekly"
    priority = 0.7

    def items(self):
        return Article.objects.published().order_by("-published_at")

    def lastmod(self, obj: Article):
        return obj.updated_at
```

### 16.11.3 Wire sitemap in `config/urls.py`
Edit `config/urls.py`:

```python
from django.contrib import admin
from django.contrib.sitemaps.views import sitemap
from django.urls import include, path

from articles.sitemaps import ArticleSitemap

sitemaps = {
    "articles": ArticleSitemap,
}

urlpatterns = [
    path("admin/", admin.site.urls),
    path("accounts/", include("django.contrib.auth.urls")),
    path("articles/", include("articles.urls")),
    path("", include("pages.urls")),
    path("sitemap.xml", sitemap, {"sitemaps": sitemaps}, name="sitemap"),
]
```

Now visit:
- `http://127.0.0.1:8000/sitemap.xml`

### 16.11.4 Add robots.txt
Create `templates/robots.txt`:

```text
User-agent: *
Disallow: /admin/
Disallow: /accounts/
Sitemap: {{ site_url }}/sitemap.xml
```

Serve it via a tiny view (TemplateView is perfect). In `pages/urls.py`:

```python
from django.views.generic import TemplateView

urlpatterns = [
    # ...
    path(
        "robots.txt",
        TemplateView.as_view(template_name="robots.txt", content_type="text/plain"),
        name="robots_txt",
    ),
]
```

Make sure `site_url` exists. If you already have `SITE_URL` in settings and a context
processor, expose it:

In `config/context_processors.py` add:

```python
from django.conf import settings

def site_meta(request):
    return {
        "site_name": getattr(settings, "SITE_NAME", "Site"),
        "site_url": getattr(settings, "SITE_URL", "").rstrip("/"),
    }
```

Register it in `TEMPLATES` context processors.

Then robots.txt will render a correct sitemap URL.

### 16.11.5 Add basic meta tags (title/description)
In `templates/base.html` add:

```django
<meta name="description" content="{% block meta_description %}A Django content site.{% endblock %}">
```

In article detail template:

```django
{% block meta_description %}
  {{ article.body|truncatechars:140 }}
{% endblock %}
```

This is basic but useful. Later you’ll implement richer OpenGraph/Twitter meta.

---

## 16.12 Tests: Prove the Whole Product Works

Create `articles/tests_project1.py`:

```python
from django.contrib.auth import get_user_model
from django.test import TestCase
from django.urls import reverse
from django.utils import timezone

from articles.models import Article, Tag


class Project1Tests(TestCase):
    def setUp(self):
        User = get_user_model()
        self.author = User.objects.create_user(
            username="author",
            password="pass12345",
            email="author@example.com",
        )
        self.tag = Tag.objects.create(name="Django", slug="django")

        self.published = Article.objects.create(
            title="Published",
            slug="published",
            body="Body " * 30,
            status=Article.Status.PUBLISHED,
            published_at=timezone.now(),
            author=self.author,
        )
        self.published.tags.add(self.tag)

        self.draft = Article.objects.create(
            title="Draft",
            slug="draft",
            body="Draft body",
            status=Article.Status.DRAFT,
            published_at=None,
            author=self.author,
        )

    def test_list_shows_only_published(self):
        url = reverse("articles:list")
        response = self.client.get(url)
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, "Published")
        self.assertNotContains(response, "Draft")

    def test_filter_by_tag(self):
        url = reverse("articles:list")
        response = self.client.get(url, {"tag": "django"})
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, "Published")

    def test_search(self):
        url = reverse("articles:list")
        response = self.client.get(url, {"q": "Published"})
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, "Published")

    def test_detail_published_visible(self):
        url = reverse("articles:detail", kwargs={"slug": "published"})
        response = self.client.get(url)
        self.assertEqual(response.status_code, 200)

    def test_detail_draft_hidden_from_anonymous(self):
        url = reverse("articles:detail", kwargs={"slug": "draft"})
        response = self.client.get(url)
        self.assertEqual(response.status_code, 404)

    def test_detail_draft_visible_to_author(self):
        self.client.login(username="author", password="pass12345")
        url = reverse("articles:detail", kwargs={"slug": "draft"})
        response = self.client.get(url)
        self.assertEqual(response.status_code, 200)
```

### Why these tests matter
They verify your most important product policy:
- drafts are not public
- authors can preview drafts
- list/search/filter behave

---

## 16.13 Final “Project 1” Checklist (Ship-Ready Basics)

### Functional
- [ ] List loads with pagination and preserves filters
- [ ] Detail shows cover image, tags, comments
- [ ] Comment submission works and is moderated
- [ ] Create/edit requires login and enforces ownership rules

### Security
- [ ] All POST forms include `{% csrf_token %}`
- [ ] Drafts are not visible publicly
- [ ] Upload size/type validation exists for cover images

### Performance baseline
- [ ] List uses `select_related("author")` and `prefetch_related("tags")`
- [ ] No obvious N+1 in templates when iterating tags

### SEO/ops
- [ ] `/sitemap.xml` works
- [ ] `/robots.txt` works
- [ ] Titles and meta descriptions exist

### Quality
- [ ] `python manage.py test` passes
- [ ] `ruff`/`black --check` pass
- [ ] admin is usable (filters/search/actions)

---

## 16.14 Review Questions (Answer without looking)

1. Why do list filters use GET query params instead of POST?
2. What causes duplicates in many-to-many queries and why do we use `.distinct()`?
3. What is the PRG pattern and why is it important for comments and article forms?
4. Why is comment moderation a security/stability feature, not “just UX”?
5. What does `prefetch_related("tags")` prevent in templates?

---

## Next Step
Next chapter is **17. Project 2 (Intermediate): Multi-User App (Tasks/CRM/Inventory)**, where you’ll learn:
- role-based access control in practice
- per-user data ownership at scale
- audit fields (created_by/updated_by)
- filtering/search UI patterns
- CSV export and internal tooling patterns

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../2. Core_Django/15. email_and_notifications.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='17. team_tasks.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
